# Analysis of Abl mutations in Perses

Below is a first draft of some inital analysis of `4xey` mutations.

In [1]:
from perses.analysis.analysis import Analysis
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymbar
%matplotlib inline
import os
import itertools
from tqdm import notebook as tqdm_notebook
import pandas as pd


from simtk.openmm import unit
from openmmtools.constants import kB
KT_KCALMOL = kB * 300 * unit.kelvin / unit.kilocalories_per_mole

In [2]:
def subtract_offset(forward_work, reverse_work):

    print("--> subtracting offset")
    
    forward_work_offset = []
    for cycle in forward_work:
        forward_work_offset.append(np.array([val - cycle[0] for val in cycle[1:]]))
    forward_work_offset = np.array(forward_work_offset)

    reverse_work_offset = []
    for cycle in reverse_work:
        reverse_work_offset.append(np.array([val - cycle[0] for val in cycle[1:]]))
    reverse_work_offset = np.array(reverse_work_offset)
    
    return forward_work_offset, reverse_work_offset

In [3]:
def analyse(forward_accumulated, reverse_accumulated):
    
    print("--> computing dg, ddg")
    dg, ddg = pymbar.bar.BAR(forward_accumulated, reverse_accumulated)
    
    return dg, ddg


In [4]:
def plot_works(forward_work_offset,
               reverse_work_offset,
               dg,
               ddg,
               phase,
               mutation,
               title,
               save=False,
               output_dir=None):
    
    CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
    
    # Plot work trajectories
    print("--> plotting work trajs")
    
    for i, cycle in enumerate(forward_work_offset):
        
        x = [(j+1)*4e-4 for j in range(len(list(cycle)))]
        y = cycle
        if i==0:
            plt.plot(x, y, color=CB_color_cycle[0], label='forward')
        else:
            plt.plot(x, y, color=CB_color_cycle[0])
        
    for i, cycle in enumerate(reverse_work_offset):
        
        x = [(j+1)*4e-4 for j in range(len(list(cycle)))]
        y = -cycle
        if i==0:
            plt.plot(x, y, color=CB_color_cycle[1], label='reverse')
        else:
            plt.plot(x, y, color=CB_color_cycle[1])
        
    plt.xlabel("$t_{neq}$ (ns)")
    plt.ylabel("work (kT)")
    plt.title(f"{title} {phase}")
    plt.legend(loc='best')
    if save:
        if output_dir is not None:
            plt.savefig(os.path.join(output_dir, f"{mutation}_{phase}_work_traj.png"), dpi=500)
            print(f"--> saved to: {os.path.join(output_dir, f'{mutation}_{phase}_work_traj.png')}")
        else:
            print("--> No output_dir specified!")
    else:
        plt.show()
    plt.clf()
    
    # Plot work distributions
    print("--> plotting work distrib")
    
    accumulated_forward = [cycle[-1] for cycle in forward_work_offset]
    accumulated_reverse = [-cycle[-1] for cycle in reverse_work_offset]
    sns.distplot(accumulated_forward, color=CB_color_cycle[0], label='forward')
    sns.distplot(accumulated_reverse, color=CB_color_cycle[1], label='reverse')
    plt.axvline(dg)
    plt.axvline(dg + ddg, linestyle='dashed')
    plt.axvline(dg - ddg, linestyle='dashed')
    plt.xlabel("work (kT)")
    plt.ylabel("p(w)")
    plt.title(f"{title} {phase}")
    plt.legend(loc='best')
    if save:
        if output_dir is not None:
            plt.savefig(os.path.join(output_dir, f"{mutation}_{phase}_work_dist.png"), dpi=500)
            print(f"--> saved to: {os.path.join(output_dir, f'{mutation}_{phase}_work_dist.png')}")
        else:
            print("--> No output_dir specified!")
    else:
        plt.show()
    plt.clf()

In [5]:
# 4xney dasatinib mutations
# mutations = ['E255K', 'E255V', 'E355A', 'F317C', 'F317I',
#              'F317L', 'F317V', 'F359C', 'F359I', 'F359V',
#              'G250E', 'H396R', 'L248R', 'L248V', 'M244V',
#              'M351T', 'T315A', 'T315I', 'V299L', 'Y253F']

# testing subset
mutations = ['E255K', 'E255V', 'E355A', 'F317C']

In [10]:
df = {}

for mutation in mutations:

    # Load and combine arrays
    forward_complex_arrays = []
    reverse_complex_arrays = []
    forward_apo_arrays = []
    reverse_apo_arrays = []

    for j in tqdm_notebook.tqdm(range(100)):

        print(f"--> job: {j}")
        forward_complex_path = f'/data/chodera/glassw/kinoml/Abl/run_neq_NoTraj/4xey/{mutation}/4xey_dasatinib_{mutation}_complex_{j}_forward.npy'
        reverse_complex_path = f'/data/chodera/glassw/kinoml/Abl/run_neq_NoTraj/4xey/{mutation}/4xey_dasatinib_{mutation}_complex_{j}_reverse.npy'
        forward_apo_path = f'/data/chodera/glassw/kinoml/Abl/run_neq_NoTraj/4xey/{mutation}/4xey_dasatinib_{mutation}_apo_{j}_forward.npy'
        reverse_apo_path = f'/data/chodera/glassw/kinoml/Abl/run_neq_NoTraj/4xey/{mutation}/4xey_dasatinib_{mutation}_apo_{j}_reverse.npy'

        if os.path.exists(forward_complex_path):
            with open(forward_complex_path, 'rb') as f:
                forward_complex_arrays.append(np.load(f))

        if os.path.exists(reverse_complex_path):
            with open(reverse_complex_path, 'rb') as f:
                reverse_complex_arrays.append(np.load(f))

        if os.path.exists(forward_apo_path):
            with open(forward_apo_path, 'rb') as f:
                forward_apo_arrays.append(np.load(f))

        if os.path.exists(reverse_apo_path):
            with open(reverse_apo_path, 'rb') as f:
                reverse_apo_arrays.append(np.load(f))

    if forward_complex_arrays and reverse_complex_arrays and forward_apo_arrays and reverse_apo_arrays:

        forward_complex_combined = np.concatenate(forward_complex_arrays)
        forward_complex_accumulated = np.array([cycle[-1] - cycle[0] for cycle in forward_complex_combined]) # compute this separately bc the last value of the subsampled array is diff than the actual last sample
        forward_complex_combined = np.array([cycle for cycle in forward_complex_combined])
        print(forward_complex_combined.shape)

        reverse_complex_combined = np.concatenate(reverse_complex_arrays)
        reverse_complex_accumulated = np.array([cycle[-1] - cycle[0] for cycle in reverse_complex_combined])
        reverse_complex_combined = np.array([cycle for cycle in reverse_complex_combined])

        forward_apo_combined = np.concatenate(forward_apo_arrays)
        forward_apo_accumulated = np.array([cycle[-1] - cycle[0] for cycle in forward_apo_combined])
        forward_apo_combined = np.array([cycle for cycle in forward_apo_combined])
        print(forward_apo_combined.shape)

        reverse_apo_combined = np.concatenate(reverse_apo_arrays)
        reverse_apo_accumulated = np.array([cycle[-1] - cycle[0] for cycle in reverse_apo_combined])
        reverse_apo_combined = np.array([cycle for cycle in reverse_apo_combined])


        # Analyse

        ## complex
        forward_complex_work_offset, reverse_complex_work_offset = subtract_offset(forward_complex_combined,
                                                                                   reverse_complex_combined)

        complex_dg, complex_ddg = analyse(forward_complex_accumulated,
                                          reverse_complex_accumulated)

        ## apo
        forward_apo_work_offset, reverse_apo_work_offset = subtract_offset(forward_apo_combined,
                                                                           reverse_apo_combined)

        apo_dg, apo_ddg = analyse(forward_apo_accumulated, reverse_apo_accumulated)

        ## plot the work trajectories and distibutions
        complex_plot = plot_works(forward_complex_work_offset,
                                  reverse_complex_work_offset,
                                  complex_dg,
                                  complex_ddg,
                                  phase='complex',
                                  mutation=mutation,
                                  title=f'4XEY-Dasatinib {mutation}',
                                  save=True,
                                  output_dir='/lila/home/glassw/GITHUB/abl_resistance/notebooks/perses_analysis_output')

        apo_plot = plot_works(forward_apo_work_offset,
                              reverse_apo_work_offset,
                              apo_dg,
                              apo_ddg,
                              phase='apo',
                              mutation=mutation,
                              title=f'4XEY-Dasatinib {mutation}',
                              save=True,
                              output_dir='/lila/home/glassw/GITHUB/abl_resistance/notebooks/perses_analysis_output')

        ## Get binding dg and ddg
        binding_dg = complex_dg - apo_dg
        binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
        df[mutation] = [binding_dg, binding_ddg]
        print(f"--> complex_dg: {complex_dg}")
        print(f"--> apo dg: {apo_dg}")

    else:
        print(f"--> dir {mutation} has at least one phase without data" )

--> job: 0
--> job: 1
--> job: 2
--> job: 3
--> job: 4
--> job: 5
--> job: 6
--> job: 7
--> job: 8
--> job: 9
--> job: 10
--> job: 11
--> job: 12
--> job: 13
--> job: 14
--> job: 15
--> job: 16
--> job: 17
--> job: 18
--> job: 19
--> job: 20
--> job: 21
--> job: 22
--> job: 23
--> job: 24
--> job: 25
--> job: 26
--> job: 27
--> job: 28
--> job: 29
--> job: 30
--> job: 31
--> job: 32
--> job: 33
--> job: 34
--> job: 35
--> job: 36
--> job: 37
--> job: 38
--> job: 39
--> job: 40
--> job: 41
--> job: 42
--> job: 43
--> job: 44
--> job: 45
--> job: 46
--> job: 47
--> job: 48
--> job: 49
--> job: 50
--> job: 51
--> job: 52
--> job: 53
--> job: 54
--> job: 55
--> job: 56
--> job: 57
--> job: 58
--> job: 59
--> job: 60
--> job: 61
--> job: 62
--> job: 63
--> job: 64
--> job: 65
--> job: 66
--> job: 67
--> job: 68
--> job: 69
--> job: 70
--> job: 71
--> job: 72
--> job: 73
--> job: 74
--> job: 75
--> job: 76
--> job: 77
--> job: 78
--> job: 79
--> job: 80
--> job: 81
--> job: 82
--> job: 83
--

--> job: 0
--> job: 1
--> job: 2
--> job: 3
--> job: 4
--> job: 5
--> job: 6
--> job: 7
--> job: 8
--> job: 9
--> job: 10
--> job: 11
--> job: 12
--> job: 13
--> job: 14
--> job: 15
--> job: 16
--> job: 17
--> job: 18
--> job: 19
--> job: 20
--> job: 21
--> job: 22
--> job: 23
--> job: 24
--> job: 25
--> job: 26
--> job: 27
--> job: 28
--> job: 29
--> job: 30
--> job: 31
--> job: 32
--> job: 33
--> job: 34
--> job: 35
--> job: 36
--> job: 37
--> job: 38
--> job: 39
--> job: 40
--> job: 41
--> job: 42
--> job: 43
--> job: 44
--> job: 45
--> job: 46
--> job: 47
--> job: 48
--> job: 49
--> job: 50
--> job: 51
--> job: 52
--> job: 53
--> job: 54
--> job: 55
--> job: 56
--> job: 57
--> job: 58
--> job: 59
--> job: 60
--> job: 61
--> job: 62
--> job: 63
--> job: 64
--> job: 65
--> job: 66
--> job: 67
--> job: 68
--> job: 69
--> job: 70
--> job: 71
--> job: 72
--> job: 73
--> job: 74
--> job: 75
--> job: 76
--> job: 77
--> job: 78
--> job: 79
--> job: 80
--> job: 81
--> job: 82
--> job: 83
--

--> job: 0
--> job: 1
--> job: 2
--> job: 3
--> job: 4
--> job: 5
--> job: 6
--> job: 7
--> job: 8
--> job: 9
--> job: 10
--> job: 11
--> job: 12
--> job: 13
--> job: 14
--> job: 15
--> job: 16
--> job: 17
--> job: 18
--> job: 19
--> job: 20
--> job: 21
--> job: 22
--> job: 23
--> job: 24
--> job: 25
--> job: 26
--> job: 27
--> job: 28
--> job: 29
--> job: 30
--> job: 31
--> job: 32
--> job: 33
--> job: 34
--> job: 35
--> job: 36
--> job: 37
--> job: 38
--> job: 39
--> job: 40
--> job: 41
--> job: 42
--> job: 43
--> job: 44
--> job: 45
--> job: 46
--> job: 47
--> job: 48
--> job: 49
--> job: 50
--> job: 51
--> job: 52
--> job: 53
--> job: 54
--> job: 55
--> job: 56
--> job: 57
--> job: 58
--> job: 59
--> job: 60
--> job: 61
--> job: 62
--> job: 63
--> job: 64
--> job: 65
--> job: 66
--> job: 67
--> job: 68
--> job: 69
--> job: 70
--> job: 71
--> job: 72
--> job: 73
--> job: 74
--> job: 75
--> job: 76
--> job: 77
--> job: 78
--> job: 79
--> job: 80
--> job: 81
--> job: 82
--> job: 83
--

--> job: 0
--> job: 1
--> job: 2
--> job: 3
--> job: 4
--> job: 5
--> job: 6
--> job: 7
--> job: 8
--> job: 9
--> job: 10
--> job: 11
--> job: 12
--> job: 13
--> job: 14
--> job: 15
--> job: 16
--> job: 17
--> job: 18
--> job: 19
--> job: 20
--> job: 21
--> job: 22
--> job: 23
--> job: 24
--> job: 25
--> job: 26
--> job: 27
--> job: 28
--> job: 29
--> job: 30
--> job: 31
--> job: 32
--> job: 33
--> job: 34
--> job: 35
--> job: 36
--> job: 37
--> job: 38
--> job: 39
--> job: 40
--> job: 41
--> job: 42
--> job: 43
--> job: 44
--> job: 45
--> job: 46
--> job: 47
--> job: 48
--> job: 49
--> job: 50
--> job: 51
--> job: 52
--> job: 53
--> job: 54
--> job: 55
--> job: 56
--> job: 57
--> job: 58
--> job: 59
--> job: 60
--> job: 61
--> job: 62
--> job: 63
--> job: 64
--> job: 65
--> job: 66
--> job: 67
--> job: 68
--> job: 69
--> job: 70
--> job: 71
--> job: 72
--> job: 73
--> job: 74
--> job: 75
--> job: 76
--> job: 77
--> job: 78
--> job: 79
--> job: 80
--> job: 81
--> job: 82
--> job: 83
--

/home/glassw/miniconda/envs/ntrk_mutations_and_perses_dev/lib/python3.7/site-packages/seaborn/distributions.py:283: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)
/home/glassw/miniconda/envs/ntrk_mutations_and_perses_dev/lib/python3.7/site-packages/seaborn/distributions.py:283: UserWarning: Data must have variance to compute a kernel density estimate.
  warnings.warn(msg, UserWarning)


--> saved to: /lila/home/glassw/GITHUB/abl_resistance/notebooks/perses_analysis_output/F317C_apo_work_dist.png
--> complex_dg: 25.816758297559495
--> apo dg: 23.726785601247446


<Figure size 432x288 with 0 Axes>

In [11]:
df

{'E255K': [0.35215018699788914, 0.46476274381559307],
 'E255V': [1.1056975785327268, 0.4407849529904972],
 'E355A': [-0.21049779665437995, 0.4432481162177818],
 'F317C': [2.0899726963120493, 0.1254895747098673]}